# Testing module

In [2]:
import csv
import os
import pandas as pd
import pickle 
import numpy as np
import mediapipe as mp
import cv2
import datetime
import joblib

from sklearn.preprocessing import StandardScaler

In [3]:
prj_path = r"C:\Users\Chale\Desktop\Project ITM65\Model Framework\Code\Web-applications-for-elderly-physical-exercises\ElderFitmodel"

In [4]:
model_name = 'handup_model'

model_dst=prj_path+"\Model/"
save_dst=model_dst+model_name+'.pkl'
model = pickle.load(open(save_dst, 'rb'))

c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.1. This might lead to brea

In [5]:
# Grabbing the Holistic Model from Mediapipe and
mp_holistic = mp.solutions.holistic

# Initializing the Model
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


In [6]:
def calculate_angle(s,e,w):
    a = np.array(s) # First
    b = np.array(e) # Mid
    c = np.array(w) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [19]:
stage = 0
count = 0

# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()

    # resizing the frame
    frame = cv2.resize(frame, (900,740))
        
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    
    results = holistic_model.process(image)
    
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,   
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        # Make Detections
        X = pd.DataFrame([pose_row])
        predict_class = model.predict(X)[0]
        predict_prob = model.predict_proba(X)[0]

        coords = tuple(np.multiply(
        np.array(
            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
             results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
        , [640,480]).astype(int))
        
        str_count = f"COUNT = {count}"
        cv2.rectangle(image, 
                      (coords[0], coords[1]+5), 
                      (coords[0]+len(str_count)*20, coords[1]-30), 
                      (245, 117, 16), -1)
        cv2.putText(image, str_count, coords, 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        if(stage == 0 and predict_class=="left_hand" and round(predict_prob[np.argmax(predict_prob)],2) >= 0.75):
            count+=1
            stage = 1
        elif(stage == 1 and predict_class=="right_hand" and round(predict_prob[np.argmax(predict_prob)],2) >= 0.75):
            count+=1
            stage = 0

        # Get status box
        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

        # Display Class
        cv2.putText(image, 'CLASS'
                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, predict_class.split(' ')[0]
                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Display Probability
        cv2.putText(image, 'PROB'
                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(predict_prob[np.argmax(predict_prob)],2))
                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    except:
        cv2.putText(image, str('Error')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
    # Display the resulting image
    cv2.imshow("Test Model", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') :
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-

Angle Version

In [6]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()

    # resizing the frame
    frame = cv2.resize(frame, (900,740))
        
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    
    results = holistic_model.process(image)
    
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,   
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        # Get s,e,w left
        shoulder_left = [pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow_left = [pose[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,pose[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
        wrist_left = [pose[mp_holistic.PoseLandmark.LEFT_WRIST.value].x,pose[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]

        # Get s,e,w right
        shoulder_right = [pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
        elbow_right = [pose[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,pose[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
        wrist_right = [pose[mp_holistic.PoseLandmark.RIGHT_WRIST.value].x,pose[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
        
        
        # Calculate angle
        angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
        angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)

        # Visualize angle left
        cv2.putText(image, str(angle_left), 
                        tuple(np.multiply(elbow_left, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
        )

        # Visualize angle right
        cv2.putText(image, str(angle_right), 
                        tuple(np.multiply(elbow_right, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
        )

        # Make Detections
        X = pd.DataFrame([pose_row])
        predict_class = model.predict(X)[0]
        predict_prob = model.predict_proba(X)[0]

        coords = tuple(np.multiply(
        np.array(
            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
             results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
        , [640,480]).astype(int))
            
        cv2.rectangle(image, 
                      (coords[0], coords[1]+5), 
                      (coords[0]+len(predict_class)*20, coords[1]-30), 
                      (245, 117, 16), -1)
        cv2.putText(image, predict_class, coords, 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        
#         # Get status box
#         cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

#         # Display Class
#         cv2.putText(image, 'CLASS'
#                     , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, predict_class.split(' ')[0]
#                     , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#         # Display Probability
#         cv2.putText(image, 'PROB'
#                     , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, str(round(predict_prob[np.argmax(predict_prob)],2))
#                     , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    except:
        cv2.putText(image, str('Error')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
    # Display the resulting image
    cv2.imshow("Test Model", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') :
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Chale\anaconda3\envs\ITMAI\lib\site-